# Data Preprocessing Notebook

# Introduction

The initial dataset being sourced is from the HuggingFace library, (https://huggingface.co/datasets/brianarbuckle/cocktail_recipes)

This initial dataset contains columns `title`, `ingredients`, `directions`, `misc`, `source`, and `ner`.

**Overview**
The 'Ingredients' column of our dataset presents a unique challenge. It consists of objects, each being a list of strings, where each string details a part of a cocktail recipe. These details typically include the quantity, unit, and name of an ingredient, but they can also contain preparation instructions or garnishing details that are not directly usable in their current form for data analysis or recipe generation.

**Objective**
Our goal is to parse these strings to extract structured information that can be effectively utilized in our project. Specifically, we aim to separate the ingredient details into distinct components: quantity, unit, and ingredient name, while filtering out the non-ingredient related information.

## Imports

In [8]:
import pandas as pd
import re
from datasets import load_dataset

pip install datasetsto access HuggingFace datasets library

In [9]:
#!pip install datasets

# Initial Data Inspection

In [10]:
# Loading the cocktail_recipes dataset
dataset = load_dataset('brianarbuckle/cocktail_recipes')

In [11]:
df = pd.DataFrame(dataset['train'])

In [12]:
df.head(5)

,title,ingredients,directions,misc,source,ner
0,151 Swizzle,[1.5 oz. 151-Proof Demerara Rum [Lemon Hart or...,[],[],Beachbum Berry Remixed,"[pernod, rum]"
1,20th Century,"[The 21st Century, 2 oz. Siete Leguas Blanco T...","[shake on ice and strain into coupe , The Best...",[],Jim Meehan,"[cocchi americano, pernod, tequila]"
2,20th Century,"[1.5 oz. Plymouth Gin, 3\/4 oz. Mari Brizard W...",[shake on ice and strain],[],PDT,"[lillet, gin]"
3,Abbey Cocktail,[],"[Shake liquid ingredients with ice., Strain in...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,[]
4,Absinthe Drip,[1 1/2 ounces Pernod (or other absinthe substi...,[Pour Pernod into a pousse-caf or sour glass....,[The Absinthe Drip was made famous by Toulouse...,The Ultimate Bar Book,"[pernod, absinthe]"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        875 non-null    object
 1   ingredients  875 non-null    object
 2   directions   875 non-null    object
 3   misc         875 non-null    object
 4   source       875 non-null    object
 5   ner          875 non-null    object
dtypes: object(6)
memory usage: 41.1+ KB


In [29]:
df.shape

(875, 6)

From this **Initial Data Inspection**, we see that the data is a 875 x 6 dataframe, entirely of objects. We also get to see that there are multiple instances of entries being `[]` constiting non-null entries.

# Data Cleaning

Section involves removing empty rows, rows irrelevant to the analysis, or any rows unusable in the analysis.

In [14]:
# Removing rows where `ingredients` is empty
non_empty_ingredients_df = df[df['ingredients'].apply(lambda x: len(x) > 0)]
modified_df = non_empty_ingredients_df.reset_index(drop=True)

In [15]:
modified_df.shape

(873, 6)

From this, we can see that the dataframe went from 875 → 873 rows. So only two rows contained no information in the ingredients section. 

# Parsing 'Ingredients'

This section will involve parsing the 'Ingredients' column in order to obtain data relevant to the project, and organize it such that it can be utilized.

For example, the current 'ingredients' column contains data such as:

In [27]:
modified_df['ingredients'][0]

['1.5 oz. 151-Proof Demerara Rum [Lemon Hart or El Dorado]',
 '0.5 oz. Lime Juice',
 '0.5 oz. Sugar Syrup',
 '1 Dash Angostura Bitters',
 '6 drops [1\\/8 Tsp.] Pernod',
 '8 oz. crushed ice',
 'blend at high speed for 5 seconds, pour into metal cup or a pilsner glass',
 'Garnish with dust of nutmeg and a cinnamon stick']

## Analysis

- **Ingredient Entries**: Most strings follow a common format: `[quantity] [unit] [ingredient]`, which allows for standardized extraction. For example, `1.5 oz. 151-Proof Demerara Rum [Lemon Hart or El Dorado]` contains clear quantity, unit, and ingredient name components.

- **Non-Ingredient Instructions**: Some strings contain instructions or garnishing details, such as `blend at high speed for 5 seconds, pour into metal cup or a pilsner glass`, which do not fit the standard ingredient format and are thus considered unusable for direct ingredient analysis.


## Strategy
To tackle this, we employ a parsing function that utilizes regular expressions to identify and extract the quantity, unit, and ingredient name from each string, while discarding any non-ingredient information. This function is designed to handle the common format efficiently and includes error handling for strings that do not conform to the expected pattern.

By parsing the 'Ingredients' column in this manner, we aim to create a structured and usable dataset that will serve as a solid foundation for the subsequent stages of our project, including data analysis and the generation of new cocktail recipes.

In [33]:
def parse_ingredient(ingredient_str):
    # First, pre-process to correct known formatting issues with fractions
    corrected_str = re.sub(r'\\', '', ingredient_str)  # Remove backslashes that might interfere with parsing
    
    # regex patterns to extact ingredients from strings
    pattern = re.compile(
        r'(?P<quantity>\d+\s*\d*\/\d+|\d*\.\d+|\d+)?\s*'  # Capture quantities, fractions, decimals
        r'(?P<unit>oz|ounces?|tsp|teaspoons?|tablespoons?|tbl|tbs|cups?|pints?|quarts?|gallons?|lbs?|pounds?|ml|mL|liters?|dash|dashes|drops?|pinch|pinches)?\.?\s*'
        r'(?P<ingredient>.+)', re.IGNORECASE)

    match = pattern.match(corrected_str)
    
    if match:
        # Normalize unit names
        unit = match.group('unit')
        if unit:
            unit = unit.lower()
            if unit in ['tsp', 'teaspoons']:
                unit = 'tsp'
            elif unit in ['tbl', 'tbs', 'tablespoons']:
                unit = 'tbsp'
            elif unit in ['pounds', 'pound', 'lbs']:
                unit = 'lb'
            elif unit in ['gallons', 'gallon']:
                unit = 'gal'
            elif unit in ['ounces', 'ounce', 'oz']:
                unit = 'oz'
            elif unit in ['milliliters', 'millilitre', 'ml', 'mL']:
                unit = 'ml'
            elif unit in ['liters', 'litre']:
                unit = 'l'
        
        return {'quantity': match.group('quantity'), 'unit': unit, 'ingredient': match.group('ingredient').strip()}
    else:
        # Handling cases that don't fit the expected pattern
        return {'quantity': None, 'unit': None, 'ingredient': corrected_str}

In [18]:
parsed_ingredients_list = []

In [34]:
for index, row in modified_df.iterrows():
    for ingredient in row['ingredients']:
        parsed_ingredient = parse_ingredient(ingredient)
        parsed_ingredient['recipe_id'] = index  # Keep track of the recipe ID
        parsed_ingredients_list.append(parsed_ingredient)

parsed_df = pd.DataFrame(parsed_ingredients_list)

We have created `parsed_df` which contains the parsed versions of the `ingredients` column of the original dataset.

In [20]:
parsed_df.shape

(3722, 4)

In [21]:
parsed_df.head(50)

,quantity,unit,ingredient,recipe_id
0,1.5,oz,151-Proof Demerara Rum [Lemon Hart or El Dorado],0
1,0.5,oz,Lime Juice,0
2,0.5,oz,Sugar Syrup,0
3,1,dash,Angostura Bitters,0
4,6,drops,[1/8 Tsp.] Pernod,0
5,8,oz,crushed ice,0
6,None,None,"blend at high speed for 5 seconds, pour into m...",0
7,None,None,Garnish with dust of nutmeg and a cinnamon stick,0
8,None,None,The 21st Century,1
9,2,oz,Siete Leguas Blanco Tequila,1


Looking into remaining edge cases, if something appears in the following, that means the quantity or unit is not successfully integrated by the parsing above.

In [23]:
# Filter rows where either 'quantity' or 'unit' is None (NaN)
none_rows = parsed_df.loc[parsed_df['quantity'].isnull() | parsed_df['unit'].isnull()]

# Display these rows
print(none_rows)

     quantity  unit                                         ingredient  \
6        None  None  blend at high speed for 5 seconds, pour into m...   
7        None  None   Garnish with dust of nutmeg and a cinnamon stick   
8        None  None                                   The 21st Century   
12       None  None                      rinse coupe glass with Pernod   
18          1  None                                         sugar cube   
...       ...   ...                                                ...   
3713     None  None                                  2oz. 90 Proof Rye   
3718     None  None                           shake on ice and strain"   
3719     None  None                                          champagne   
3720     None  None                                   orange juice-ish   
3721     None  None                  optional but preferable 1 oz. gin   

      recipe_id  
6             0  
7             0  
8             1  
12            1  
18            3  
...